### Titanic Classification - with Ensemble ###

Purpose: Inspired by the Kaggle Titanic Competition and wanting to learn about multi-model 'ensembles', I set to work trying to beat my own top classification accuracy score of ~81% on Titanic classification (seriously, that should get its own class in SciKit Learn! {side note, if you "got" that joke, you are as big a nerd as I am - and that is a compliment! ;) })

`[Spoiler Alert: I was able to get ~85% accuracy on the test-set!]`

*This is the simplified, single pass using and ensemble of three algos combined in a VotingClassifier (my first time using one of these - fascinating!) --> I built another notebook which uses GridSearchCV to fine-tune hyper-params and then use the VotingClassifier)

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

import warnings 
warnings.simplefilter('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [13]:
# Load the Titanic dataset (note: this dataset is from Kaggle, as I wanted to stick to the contest params, but while it is called 'train' it does have all 891 rows of data.)

df = pd.read_csv('../data/titanic_data/train.csv')

df.shape
df.head(3)
df.info()

(891, 12)

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
# Preprocess the dataset (set up as a function bc I've preprocessing the darn dataset so many times already!)

def preprocess_data(df):
    # Fill missing values
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)

    # Convert categorical features to numeric
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

    # Drop irrelevant columns
    df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    return df

# Preprocess the data
df = preprocess_data(df)

In [6]:
df.shape
df.head(3)
df.info()

(891, 8)

Survived  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked
0         0       3    0  22.0      1      0   7.2500         2
1         1       1    1  38.0      1      0  71.2833         0
2         1       3    1  26.0      0      0   7.9250         2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int64  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 55.8 KB


In [7]:
# Separate features and target variable
X = df.drop('Survived', axis=1)
y = df['Survived']

X.shape, y.shape

((891, 7), (891,))

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2
    , random_state=27
)

# Display the shapes of the training and test sets
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 7), (179, 7), (712,), (179,))

In [9]:
# Initialize ensemble models (*Note: Since these are all tree methods, we did NOT standardize;)

rf_model = RandomForestClassifier(
    n_estimators=100, random_state=27
)
gb_model = GradientBoostingClassifier(
    n_estimators=100, random_state=27
)
xgb_model = XGBClassifier(
    n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=27
)

In [10]:
# Fit models individually

rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)

RandomForestClassifier(random_state=27)

GradientBoostingClassifier(random_state=27)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [11]:
# Make predictions
rf_preds = rf_model.predict(X_test)
gb_preds = gb_model.predict(X_test)
xgb_preds = xgb_model.predict(X_test)

# Evaluate individual models
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print("Gradient Boosting Accuracy:", accuracy_score(y_test, gb_preds))
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))

Random Forest Accuracy: 0.8268156424581006
Gradient Boosting Accuracy: 0.8379888268156425
XGBoost Accuracy: 0.8044692737430168


In [12]:
# Voting Classifier (Ensemble of all three models)

voting_model = VotingClassifier(
    estimators=[
        ('rf', rf_model)
        , ('gb', gb_model)
        , ('xgb', xgb_model)
    ], voting='soft'
)
voting_model.fit(X_train, y_train)
voting_preds = voting_model.predict(X_test)

# Evaluate Voting Classifier
print("Voting Classifier Accuracy:", accuracy_score(y_test, voting_preds))

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=27)),
                             ('gb',
                              GradientBoostingClassifier(random_state=27)),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric='logloss',
                                            featu...
                                            feature_weights=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None, ...))],
                 voting='soft')

Voting Classifier Accuracy: 0.8491620111731844
